In [1]:
import requests
import json
import pandas as pd
from time import sleep

In [2]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Emilia Romagna
df_er = df_comuni[df_comuni['cod_regione'] == '08']

In [3]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

/tmp/ipykernel_1879/3372484651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url_aff'] = df_er.apply(
/tmp/ipykernel_1879/3372484651.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url'] = df_er.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune,url_aff,url
192,080560560300,ROCCABIANCA,CM,N,NaN,7,08,EMILIA-ROMAGNA,056,PARMA,056,0300,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
161,080500500460,ZOCCA,CM,N,NaN,7,08,EMILIA-ROMAGNA,050,MODENA,050,0460,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
245,080610610380,RIVERGARO,CM,N,NaN,7,08,EMILIA-ROMAGNA,061,PIACENZA,061,0380,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...


In [4]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [5]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff']):
   print(f"\nProcessing record {idx + 1}")
   print(f"URL: {url}")

   response = requests.get(url, headers=headers)
   print(f"Response status code: {response.status_code}")
   
   data = response.json()

   comune = df_er.iloc[idx]['desc']
   provincia = df_er.iloc[idx]['provincia']
   regione = df_er.iloc[idx]['regione']
   url = df_er.iloc[idx]['url']

   print(f"Processing data for Comune: {comune}, Provincia: {provincia}")

   # Start with basic comune info
   comune_info = {
       'comune': comune,
       'provincia': provincia,
       'regione': regione,
       'url': url
   }
   
   # Add candidate totals
   if 'cand' in data:
       for cand in data['cand']:
           nome_completo = cand['nome'] + " " + cand['cogn']
           comune_info[f"{nome_completo}_total"] = cand['voti']
           comune_info[f"{nome_completo}_perc"] = cand['perc']
           print(f"{nome_completo}: {cand['voti']} votes ({cand['perc']}%)")

   comuni_data.append(comune_info)
   
   print(f"Processed {comune}")
   sleep(0.1)

# Create DataFrame
print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nSample of processed data:")
print(df_data.head())


Processing record 1
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/013/CM/0005


Response status code: 200
Processing data for Comune: ALTO RENO TERME, Provincia: BOLOGNA
MICHELE DE PASCALE: 1240 votes (49,13%)
ELENA UGOLINI: 1204 votes (47,70%)
FEDERICO SERRA: 59 votes (2,34%)
LUCA TEODORI: 21 votes (0,83%)
Processed ALTO RENO TERME

Processing record 2
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/013/CM/0010


Response status code: 200
Processing data for Comune: ANZOLA DELL'EMILIA, Provincia: BOLOGNA
MICHELE DE PASCALE: 3346 votes (66,04%)
ELENA UGOLINI: 1561 votes (30,81%)
FEDERICO SERRA: 97 votes (1,91%)
LUCA TEODORI: 63 votes (1,24%)
Processed ANZOLA DELL'EMILIA

Processing record 3
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/013/CM/0020


Response status code: 200
Processing data for Comune: ARGELATO, Provincia: BOLOGNA
MICHELE DE PASCALE: 2536 votes (63,51%)
ELENA UGOLINI: 1327 votes (33,23%)
FEDERICO SERRA: 75 votes (1,88%)
LUCA TEODORI: 55 votes (1,38%)
Processed ARGELATO

Processing record 4
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/013/CM/0030


Response status code: 200
Processing data for Comune: BARICELLA, Provincia: BOLOGNA
MICHELE DE PASCALE: 1277 votes (51,60%)
ELENA UGOLINI: 1115 votes (45,05%)
FEDERICO SERRA: 42 votes (1,70%)
LUCA TEODORI: 41 votes (1,66%)
Processed BARICELLA

Processing record 5
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/013/CM/0050


...

Response status code: 200
Processing data for Comune: SANT'AGATA FELTRIA, Provincia: RIMINI
MICHELE DE PASCALE: 363 votes (52,53%)
ELENA UGOLINI: 313 votes (45,30%)
LUCA TEODORI: 10 votes (1,45%)
FEDERICO SERRA: 5 votes (0,72%)
Processed SANT'AGATA FELTRIA

Processing record 327
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/101/CM/0180


Response status code: 200
Processing data for Comune: SANTARCANGELO DI ROMAGNA, Provincia: RIMINI
MICHELE DE PASCALE: 5130 votes (61,83%)
ELENA UGOLINI: 2916 votes (35,15%)
LUCA TEODORI: 163 votes (1,96%)
FEDERICO SERRA: 88 votes (1,06%)
Processed SANTARCANGELO DI ROMAGNA

Processing record 328
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/101/CM/0183


Response status code: 200
Processing data for Comune: SASSOFELTRIO, Provincia: RIMINI
ELENA UGOLINI: 262 votes (61,65%)
MICHELE DE PASCALE: 152 votes (35,76%)
LUCA TEODORI: 6 votes (1,41%)
FEDERICO SERRA: 5 votes (1,18%)
Processed SASSOFELTRIO

Processing record 329
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/101/CM/0185


Response status code: 200
Processing data for Comune: TALAMELLO, Provincia: RIMINI
MICHELE DE PASCALE: 185 votes (48,56%)
ELENA UGOLINI: 184 votes (48,29%)
LUCA TEODORI: 9 votes (2,36%)
FEDERICO SERRA: 3 votes (0,79%)
Processed TALAMELLO

Processing record 330
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/101/CM/0200


Response status code: 200
Processing data for Comune: VERUCCHIO, Provincia: RIMINI
MICHELE DE PASCALE: 1662 votes (50,09%)
ELENA UGOLINI: 1507 votes (45,42%)
LUCA TEODORI: 116 votes (3,50%)
FEDERICO SERRA: 33 votes (0,99%)
Processed VERUCCHIO

Creating DataFrame...

Sample of processed data:
               comune provincia         regione  \
0     ALTO RENO TERME   BOLOGNA  EMILIA-ROMAGNA   
1  ANZOLA DELL'EMILIA   BOLOGNA  EMILIA-ROMAGNA   
2            ARGELATO   BOLOGNA  EMILIA-ROMAGNA   
3           BARICELLA   BOLOGNA  EMILIA-ROMAGNA   
4         BENTIVOGLIO   BOLOGNA  EMILIA-ROMAGNA   

                                                 url  \
0  https://elezioni.interno.gov.it/risultati/2024...   
1  https://elezioni.interno.gov.it/risultati/2024...   
2  https://elezioni.interno.gov.it/risultati/2024...   
3  https://elezioni.interno.gov.it/risultati/2024...   
4  https://elezioni.interno.gov.it/risultati/2024...   

   MICHELE DE PASCALE_total MICHELE DE PASCALE_perc  ELENA UGOL

In [6]:
df_data

,comune,provincia,regione,url,MICHELE DE PASCALE_total,MICHELE DE PASCALE_perc,ELENA UGOLINI_total,ELENA UGOLINI_perc,FEDERICO SERRA_total,FEDERICO SERRA_perc,LUCA TEODORI_total,LUCA TEODORI_perc
0,ALTO RENO TERME,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,1240,"49,13",1204,"47,70",59,"2,34",21,"0,83"
1,ANZOLA DELL'EMILIA,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,3346,"66,04",1561,"30,81",97,"1,91",63,"1,24"
2,ARGELATO,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,2536,"63,51",1327,"33,23",75,"1,88",55,"1,38"
3,BARICELLA,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,1277,"51,60",1115,"45,05",42,"1,70",41,"1,66"
4,BENTIVOGLIO,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,1357,"60,15",806,"35,73",54,"2,39",39,"1,73"
...,...,...,...,...,...,...,...,...,...,...,...,...
325,SANT'AGATA FELTRIA,RIMINI,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,363,"52,53",313,"45,30",5,"0,72",10,"1,45"
326,SANTARCANGELO DI ROMAGNA,RIMINI,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,5130,"61,83",2916,"35,15",88,"1,06",163,"1,96"
327,SASSOFELTRIO,RIMINI,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,152,"35,76",262,"61,65",5,"1,18",6,"1,41"
328,TALAMELLO,RIMINI,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,185,"48,56",184,"48,29",3,"0,79",9,"2,36"


In [7]:
# df_data['perc_diff'] = round(((df_data['MICHELE DE PASCALE_total'] - df_data['ELENA UGOLINI_total']) / df_data['ELENA UGOLINI_total']) * 100, 2)

# Get all columns containing '_total'
total_columns = [col for col in df_data.columns if '_total' in col]

# Sum all vote totals for each comune
df_data['total_votes'] = df_data[total_columns].sum(axis=1)

# Calculate the spectrum considering all votes
df_data['vote_spectrum'] = round((df_data['MICHELE DE PASCALE_total'] / df_data['total_votes']) * 100, 2)

In [8]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data.head(2)

,comune,provincia,regione,url,MICHELE DE PASCALE_total,MICHELE DE PASCALE_perc,ELENA UGOLINI_total,ELENA UGOLINI_perc,FEDERICO SERRA_total,FEDERICO SERRA_perc,LUCA TEODORI_total,LUCA TEODORI_perc,total_votes,vote_spectrum
0,Alto Reno Terme,Bologna,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,1240,"49,13",1204,"47,70",59,"2,34",21,"0,83",2524,49.13
1,Anzola Dell'Emilia,Bologna,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,3346,"66,04",1561,"30,81",97,"1,91",63,"1,24",5067,66.04


In [9]:
# Remove rows where 'comune' is either 'Sassofeltrio' or 'Montecopiolo'
df_data = df_data[~df_data['comune'].isin(['Sassofeltrio', 'Montecopiolo'])]

# Use .str.replace() to handle string replacement more safely
df_data['comune'] = df_data['comune'].str.replace("Forli'", "Forlì", regex=False)
df_data['comune'] = df_data['comune'].str.replace("Montescudo - Monte Colombo", "Montescudo-Monte Colombo", regex=False)

/tmp/ipykernel_1879/1802013088.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['comune'] = df_data['comune'].str.replace("Forli'", "Forlì", regex=False)
/tmp/ipykernel_1879/1802013088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['comune'] = df_data['comune'].str.replace("Montescudo - Monte Colombo", "Montescudo-Monte Colombo", regex=False)


In [10]:
df_data.to_csv('../output/risultati_er.csv', index=False, encoding='UTF-8-sig')